# Why GT4Py?

# Example 1: A simple point-wise stencil

First, we compare a NumPy, CuPy and GT4Py implementation of the point-wise stencil
```
d[i, j, k] = a[i, j, k] + b[i, j, k] - c[i, j, k]
```
Timings are measured using the `%timeit` magic command. This command times a single statement or function call by averaging over multiple runs. It has the additional advantage of synchronizing the CPU and the GPU at the end of each run, thus making the output reliable.

### NumPy

In [1]:
import numpy as np

shape = (512, 512, 128)

def f_numpy(a, b, c, d):
    d[...] = a + b - c
    
a = np.random.rand(*shape)
b = np.random.rand(*shape)
c = np.random.rand(*shape)
d = np.empty_like(a)

%timeit f_numpy(a, b, c, d)

197 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### CuPy

In [2]:
import cupy as cp

shape = (512, 512, 128)

def f_cupy(a, b, c, d):
    d[...] = a + b - c
    
a = cp.asarray(np.random.rand(*shape))
b = cp.asarray(np.random.rand(*shape))
c = cp.asarray(np.random.rand(*shape))
d = cp.empty_like(a)

%timeit f_cupy(a, b, c, d)

3.92 ms ± 45.8 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### GT4Py

In [3]:
import gt4py as gt
from gt4py import gtscript
import numpy as np

backend = "cuda"
shape = (512, 512, 128)

@gtscript.stencil(backend=backend)
def f_gt4py(
    a: gtscript.Field[float], 
    b: gtscript.Field[float], 
    c: gtscript.Field[float], 
    d: gtscript.Field[float]
):
    with computation(PARALLEL), interval(...):
        d = a + b - c
        
a_np = np.random.rand(*shape)
b_np = np.random.rand(*shape)
c_np = np.random.rand(*shape)

a = gt.storage.from_array(a_np, backend, (0, 0, 0))
b = gt.storage.from_array(b_np, backend, (0, 0, 0))
c = gt.storage.from_array(c_np, backend, (0, 0, 0))
d = gt.storage.empty(backend, (0, 0, 0), shape, float)

# copy data to GPU memory
f_gt4py(a=a, b=b, c=c, d=d, origin=(0, 0, 0), domain=shape)

%timeit f_gt4py(a=a, b=b, c=c, d=d, origin=(0, 0, 0), domain=shape)

/users/class151/HPC4WC_venv/lib/python3.9/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


2.04 ms ± 499 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Example 2: Laplacian

The next example compares a NumPy, CuPy and GT4Py implementation of the Laplacian stencil that we saw before in our stencil2d code that we have been working with:
```
lap[i, j, k] = - 4 * phi[  i,   j, k] 
               +     phi[i-1,   j, k] 
               +     phi[i+1,   j, k] 
               +     phi[  i, j-1, k] 
               +     phi[  i, j+1, k]
```
Timings are measured using the `%timeit` magic command. This command times a single statement or function call by averaging over multiple runs. It has the additional advantage of synchronizing the CPU and the GPU at the end of each run, thus making the output reliable.

### NumPy

In [4]:
import numpy as np

shape = (512, 512, 128)

def lap_numpy(phi, lap):
    lap[1:-1, 1:-1] = (
        - 4. * phi[1:-1, 1:-1]
        +      phi[ :-2, 1:-1]
        +      phi[  2:, 1:-1]
        +      phi[1:-1,  :-2]
        +      phi[1:-1,   2:]
    )
    
phi = np.random.rand(*shape)
lap = np.empty_like(phi)

%timeit lap_numpy(phi, lap)

311 ms ± 246 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### CuPy

In [5]:
import cupy as cp

shape = (512, 512, 128)

def lap_cupy(phi, lap):
    lap[1:-1, 1:-1] = (
        - 4. * phi[1:-1, 1:-1]
        +      phi[ :-2, 1:-1]
        +      phi[  2:, 1:-1]
        +      phi[1:-1,  :-2]
        +      phi[1:-1,   2:]
    )
    
phi = cp.asarray(np.random.rand(*shape))
lap = cp.empty_like(phi)

%timeit lap_cupy(phi, lap)

7.74 ms ± 74.4 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### GT4Py

In [6]:
import gt4py as gt
from gt4py import gtscript
import numpy as np

backend = "cuda"
shape = (512, 512, 128)

@gtscript.stencil(backend=backend, verbose=True)
def lap_gt4py(phi: gtscript.Field[float], lap: gtscript.Field[float]):
    with computation(PARALLEL), interval(...):
        lap = (
            - 4. * phi[ 0,  0, 0]
            +      phi[-1,  0, 0]
            +      phi[+1,  0, 0]
            +      phi[ 0, -1, 0]
            +      phi[ 0, +1, 0]
        )
        
phi_np = np.random.rand(*shape)
phi = gt.storage.from_array(phi_np, backend, (1, 1, 0))
lap = gt.storage.empty(backend, (1, 1, 0), shape, float)

domain = (shape[0] - 2, shape[1] - 2, shape[2])

# copy data to GPU memory
lap_gt4py(phi=phi, lap=lap, origin=(1, 1, 0), domain=domain)

%timeit lap_gt4py(phi=phi, lap=lap, origin=(1, 1, 0), domain=domain)

1.21 ms ± 61.4 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
